In [1]:
import sys
sys.path.append("../")

import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformers import BitsAndBytesConfig

print("cuda.is_available", torch.cuda.is_available())
# torch.set_default_device("cuda")

# inference_dtype = torch.float32
# inference_dtype = torch.float16
inference_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             torch_dtype=inference_dtype,
                                             device_map = "cuda:0",
                                             load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

model_tl = HookedTransformer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             hf_model=model,
                                             dtype=inference_dtype,
                                             fold_ln=False,
                                             fold_value_biases=False,
                                             center_writing_weights=False,
                                             center_unembed=False,
                                             tokenizer=tokenizer)

model_tl.generate("The capital of Germany is", max_new_tokens=2, temperature=0)

cuda.is_available True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer


  0%|          | 0/2 [00:00<?, ?it/s]

'The capital of Germany is Berlin.'

In [4]:
import torch
print("free(Gb):", torch.cuda.mem_get_info()[0]/1000000000, "total(Gb):", torch.cuda.mem_get_info()[1]/1000000000)

free(Gb): 5.531500544 total(Gb): 11.721637888
